# Import libraries

In [2]:
import requests
import pandas as pd

# Part 1: Extract, Transform, Load (ETL)

## 1.1 Extract Data
The dataset contains a list of top-level domain names, their types, and their sponsoring organization. For more information on the dataset, see https://datahub.io/core/top-level-domain-names


In [10]:
def extract_data(url, file_name): 
    # Send a GET request to the URL
    response = requests.get(url)

    # Check first if the request was successful 
    # Write the content to a file if successful
    if response.status_code == 200:
        with open(file_name, "wb") as file:
            file.write(response.content)
        print("Success! File downloaded")
    else:
        print("Failed to download file:", response.status_code)

In [11]:
url = "https://datahub.io/core/top-level-domain-names/r/top-level-domain-names.csv.csv"
extract_data(url, 'ETL_data.csv')

Success! File downloaded


## 1.2 Transform Data
We desire a dataset that contains a list of **generic** top-level domains. In the following code blocks, we will be transforming the extracted data such that the dataframe only contains rows of 'generic' type

In [5]:
# read extracted data in a dataframe
df = pd.read_csv('ETL_data.csv')
df

,Domain,Type,Sponsoring Organisation
0,.abbott,generic,"Abbott Laboratories, Inc."
1,.abogado,generic,Top Level Domain Holdings Limited
2,.ac,country-code,Network Information Center (AC Domain Registry...
3,.academy,generic,"Half Oaks, LLC"
4,.accountant,generic,dot Accountant Limited
...,...,...,...
914,.zip,generic,Charleston Road Registry Inc.
915,.zm,country-code,Zambia Information and Communications Technolo...
916,.zone,generic,"Outer Falls, LLC"
917,.zuerich,generic,Kanton Zürich (Canton of Zurich)


In [9]:
# filter dataframe 
generic_df = df[df['Type'] == 'generic']
generic_df

,Domain,Type,Sponsoring Organisation
0,.abbott,generic,"Abbott Laboratories, Inc."
1,.abogado,generic,Top Level Domain Holdings Limited
3,.academy,generic,"Half Oaks, LLC"
4,.accountant,generic,dot Accountant Limited
5,.accountants,generic,"Knob Town, LLC"
...,...,...,...
910,.yokohama,generic,"GMO Registry, Inc."
911,.youtube,generic,Charleston Road Registry Inc.
914,.zip,generic,Charleston Road Registry Inc.
916,.zone,generic,"Outer Falls, LLC"


In [ ]:
from datetime import datetime, date

today = date.today()

# Insert a new column named 'Date', so that other users would know when this data was extracted
generic_df['Date'] = today.strftime('%Y-%m-%d') 

In [14]:
generic_df

,Domain,Type,Sponsoring Organisation,Date
0,.abbott,generic,"Abbott Laboratories, Inc.",2024-02-19
1,.abogado,generic,Top Level Domain Holdings Limited,2024-02-19
3,.academy,generic,"Half Oaks, LLC",2024-02-19
4,.accountant,generic,dot Accountant Limited,2024-02-19
5,.accountants,generic,"Knob Town, LLC",2024-02-19
...,...,...,...,...
910,.yokohama,generic,"GMO Registry, Inc.",2024-02-19
911,.youtube,generic,Charleston Road Registry Inc.,2024-02-19
914,.zip,generic,Charleston Road Registry Inc.,2024-02-19
916,.zone,generic,"Outer Falls, LLC",2024-02-19


In [15]:
generic_df.to_csv('transformed_data.csv', index=False)

## 1.3 Load Data
We will store the transformed CSV file into a SQLite database. Using the `sqlalchemy` library, we can connect and write to a SQLite database

In [16]:
df = pd.read_csv('ELT_transformed_data.csv')

In [17]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///database.db')

# Write the dataframe to a database table named 'domain_data'
df.to_sql('domain_data', engine, if_exists='replace', index=False)

590

# Part 2: Extract, Load, and Transform (ELT)

## 2.1 Extract Data
Call the defined function in Part 1 to extract data from the specified URL. For more information on the dataset, see https://datahub.io/core/world-cities.

In [12]:
url = 'https://pkgstore.datahub.io/core/world-cities/world-cities_csv/data/6cc66692f0e82b18216a48443b6b95da/world-cities_csv.csv'
extract_data(url, 'ELT_data.csv')

Success! File downloaded


In [23]:
df = pd.read_csv('ELT_data.csv')
df

,name,country,subcountry,geonameid
0,les Escaldes,Andorra,Escaldes-Engordany,3040051
1,Andorra la Vella,Andorra,Andorra la Vella,3041563
2,Umm al Qaywayn,United Arab Emirates,Umm al Qaywayn,290594
3,Ras al-Khaimah,United Arab Emirates,Raʼs al Khaymah,291074
4,Khawr Fakkān,United Arab Emirates,Ash Shāriqah,291696
...,...,...,...,...
23013,Bulawayo,Zimbabwe,Bulawayo,894701
23014,Bindura,Zimbabwe,Mashonaland Central,895061
23015,Beitbridge,Zimbabwe,Matabeleland South,895269
23016,Epworth,Zimbabwe,Harare,1085510


## 2.2 Load Data

In [24]:
# Write the dataframe to a database table named 'major_cities_data'
df.to_sql('major_cities_data', engine, if_exists='replace', index=False)

23018

## 2.3 Transform Data
For this particular data, we want to filter the table such that it retains only the cities (rows) in which the country names start with letters A to M.

In [32]:
import sqlite3

# Connect to the SQLite database specfied
conn = sqlite3.connect('database.db')

# Create a cursor object
cursor = conn.cursor()

In [33]:
# Create the SQL query
query = """
        DELETE FROM major_cities_data
        WHERE country NOT BETWEEN 'A' AND 'M'
        """

# Execute the query
cursor.execute(query)

# Save changes
conn.commit()

In [34]:
# Close the cursor and connection
cursor.close()
conn.close()